In [1]:
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
# dataframe = pandas.read_csv("kddforpandatrain.csv")#, header=True)
dataframe = pandas.read_csv("kdd_dataset.csv")#, header=True)  # read the whole 10% dataset into dataframe

# samples 3000 random data points from 500k
dataframe = dataframe.sample(n=30)

# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)

dataset = dataframe_encoded.values

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]

init_offset = 0


In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors


4


In [5]:
#TODO: Vary and log our data using this
def set_and_log_hyperparams(offset):
    [i, h1, h2, h3, o] = [41, 19 + offset, 5 + offset, 0 + offset, num_of_classes] # add offset to relevant node counts
    f = open('KerasOptimizationResults.txt','a+')
    f.write("inputs: " + str(i) + ", hidden1: " + str(h1) + ", hidden2: " + str(h2) + ", hidden3: " + str(h3) + ", outputs: " + str(o))
    f.close()
    return [i, h1, h2, h3, o] # return the values

In [6]:
# define baseline model
def baseline_model(offset):
    # create model
    model = Sequential()
    
    
    inputs = 41
    hidden_layer1 = 19
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    [inputs, hidden_layer1, hidden_layer2, hidden_layer3, outputs] = set_and_log_hyperparams(offset)
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
#build a wrapper class that takes no args
def get_model():
    model = baseline_model(init_offset + 1)
    return model

In [8]:
offset = 0
for i in range(0, 10):
    estimator = KerasClassifier(build_fn=get_model, epochs=200, batch_size=5, verbose=0)
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(estimator, X, dummy_y, cv=kfold)
    result_string = "Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)
    f = open('KerasOptimizationResults.txt','a+')
    f.write("      Results: " + result_string + "\n")
    f.close()
    print(result_string)

Baseline: 83.33% (22.36%)
Baseline: 63.33% (31.45%)
Baseline: 60.00% (29.06%)
Baseline: 76.67% (26.03%)
Baseline: 66.67% (29.81%)
Baseline: 53.33% (33.99%)
Baseline: 66.67% (25.82%)
Baseline: 63.33% (34.80%)
Baseline: 73.33% (24.94%)
Baseline: 60.00% (32.66%)


In [ ]:
# this didn't work at all Cause we're doing kfold